<a href="https://colab.research.google.com/github/sanjanasrinivas22/1BM23CS301-AI/blob/main/Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
print("Sanjana Srinivas-1BM23CS301")
import re
import copy

# ============================================================
# Predicate Class (fixed to normalize multiple leading '~')
# ============================================================
class Predicate:
    def __init__(self, predicate_string):
        self.predicate_string = predicate_string.strip()
        self.name, self.arguments, self.negative = self.parse_predicate(self.predicate_string)

    def parse_predicate(self, predicate_string):
        # normalize leading tildes: an even number -> positive, odd -> negative
        s = predicate_string.strip()
        neg = False
        # strip and toggle neg for each leading '~'
        while s.startswith('~'):
            neg = not neg
            s = s[1:].strip()
        m = re.match(r"^([A-Za-z_][A-Za-z0-9_]*)\((.*?)\)$", s)
        if not m:
            raise ValueError(f"Invalid predicate: {predicate_string}")
        name, args = m.groups()
        args = [a.strip() for a in args.split(",") if a.strip() != ""]
        return name, args, neg

    def negate(self):
        self.negative = not self.negative
        self.predicate_string = ("~" if self.negative else "") + self.name + "(" + ",".join(self.arguments) + ")"

    def unify_with_predicate(self, other):
        if self.name != other.name or len(self.arguments) != len(other.arguments):
            return False
        subs = {}

        def is_var(x):
            return len(x) > 0 and x[0].islower()

        for a, b in zip(self.arguments, other.arguments):
            if a == b:
                continue
            if is_var(a) and is_var(b):
                if a in subs and subs[a] != b:
                    return False
                if b in subs and subs[b] != a:
                    return False
                subs[a] = b
            elif is_var(a):
                if a in subs and subs[a] != b:
                    return False
                subs[a] = b
            elif is_var(b):
                if b in subs and subs[b] != a:
                    return False
                subs[b] = a
            else:
                return False
        return subs

    def substitute(self, subs):
        self.arguments = [subs.get(a, a) for a in self.arguments]
        self.predicate_string = ("~" if self.negative else "") + self.name + "(" + ",".join(self.arguments) + ")"

    def __repr__(self):
        return self.predicate_string

    def __eq__(self, other):
        return isinstance(other, Predicate) and self.name == other.name and self.arguments == other.arguments and self.negative == other.negative

    def __hash__(self):
        return hash((self.name, tuple(self.arguments), self.negative))


# ============================================================
# Statement Class (CNF Clause)
# ============================================================
class Statement:
    def __init__(self, statement_string):
        self.statement_string = statement_string.strip()
        self.predicate_set = self.parse_statement(self.statement_string)

    def parse_statement(self, statement_string):
        parts = [p.strip() for p in statement_string.split('|') if p.strip() != '']
        predicates = [Predicate(p) for p in parts]
        return frozenset(predicates)

    def add_statement_to_KB(self, KB, KB_HASH):
        if self in KB:
            return
        KB.add(self)
        for predicate in self.predicate_set:
            key = predicate.name
            if key not in KB_HASH:
                KB_HASH[key] = set()
            KB_HASH[key].add(self)

    def get_resolving_clauses(self, KB_HASH):
        resolving_clauses = set()
        for predicate in self.predicate_set:
            key = predicate.name
            if key in KB_HASH:
                resolving_clauses |= KB_HASH[key]
        return resolving_clauses

    def resolve(self, other):
        new_statements = set()
        for p1 in self.predicate_set:
            for p2 in other.predicate_set:
                if p1.name == p2.name and p1.negative != p2.negative:
                    subs = p1.unify_with_predicate(p2)
                    if subs is False:
                        continue
                    new_pred_set = set()
                    for pred in (set(self.predicate_set) | set(other.predicate_set)):
                        if pred == p1 or pred == p2:
                            continue
                        pred_copy = copy.deepcopy(pred)
                        pred_copy.substitute(subs)
                        new_pred_set.add(pred_copy)
                    if not new_pred_set:
                        return False  # contradiction (empty clause)
                    sorted_preds = sorted([str(p) for p in new_pred_set])
                    new_stmt = Statement('|'.join(sorted_preds))
                    new_statements.add(new_stmt)
        return new_statements

    def __repr__(self):
        sorted_preds = sorted([str(p) for p in self.predicate_set])
        return '|'.join(sorted_preds)

    def __eq__(self, other):
        return isinstance(other, Statement) and self.predicate_set == other.predicate_set

    def __hash__(self):
        return hash(self.predicate_set)


# ============================================================
# Utility: Convert FOL Sentence to CNF Clauses (fixed negation)
# ============================================================
def fol_to_cnf_clauses(sentence):
    """
    Convert simple implications and conjunctions into CNF clauses.
    Handles antecedents that may already be negated (starting with '~').
    """
    sentence = sentence.replace(' ', '')
    if '=>' in sentence:
        lhs, rhs = sentence.split('=>')
        parts = [p for p in lhs.split('&') if p != '']
        negated_lhs = []
        for p in parts:
            # if p starts with '~', negation of ~p is p (remove ~)
            if p.startswith('~'):
                negated_lhs.append(p[1:])
            else:
                negated_lhs.append('~' + p)
        disjunction = '|'.join(negated_lhs + [rhs])
        return [disjunction]
    if '&' in sentence:
        return [c for c in sentence.split('&') if c != '']
    return [sentence]


# ============================================================
# Knowledge Base & Resolution Logic
# ============================================================
KILL_LIMIT = 8000

def prepare_knowledgebase(fol_sentences):
    KB = set()
    KB_HASH = {}
    for sentence in fol_sentences:
        clauses = fol_to_cnf_clauses(sentence)
        for clause in clauses:
            stmt = Statement(clause)
            stmt.add_statement_to_KB(KB, KB_HASH)
    return KB, KB_HASH

def FOL_Resolution(KB, KB_HASH, query, verbose=True):
    KB = set(KB)
    KB_HASH = {k: set(v) for k, v in KB_HASH.items()}

    query.add_statement_to_KB(KB, KB_HASH)
    if verbose:
        print("\nInitial KB clauses (including negated query):")
        for s in KB:
            print("  ", s)

    iterations = 0
    while True:
        iterations += 1
        if len(KB) > KILL_LIMIT:
            if verbose:
                print("Reached KILL_LIMIT, stopping.")
            return False
        new_statements = set()
        kb_list = list(KB)
        for s1 in kb_list:
            for s2 in s1.get_resolving_clauses(KB_HASH):
                if s1 == s2:
                    continue
                resolvents = s1.resolve(s2)
                if resolvents is False:
                    if verbose:
                        print(f"\nCONTRADICTION derived by resolving:\n   {s1}\n   {s2}\n=> empty clause (proof found).")
                    return True
                new_statements |= resolvents
        if new_statements.issubset(KB):
            if verbose:
                print("\nNo new clauses derived. Resolution failed (query not proved).")
            return False
        added = new_statements - KB
        if verbose and added:
            print(f"\nIteration {iterations}: Derived {len(added)} new clause(s):")
            for st in sorted(added, key=lambda x: str(x)):
                print("   ", st)
        for st in added:
            st.add_statement_to_KB(KB, KB_HASH)


# ============================================================
# Example KB (from Week-9 lab) and a goal
# ============================================================
def main():
    fol_sentences = [
        "Food(x) => Likes(John,x)",
        "Food(Apple)",
        "Food(Vegetables)",
        "Eats(p,y)&~Killed(y) => Food(y)",
        "Eats(Anil,Peanuts)",
        "Alive(Anil)",
        "Eats(Anil,y) => Eats(Harry,y)",
        "Alive(x) => ~Killed(x)",
        "~Killed(x) => Alive(x)"
    ]

    goal = "Likes(John,Peanuts)"

    KB, KB_HASH = prepare_knowledgebase(fol_sentences)

    neg_goal_pred = Predicate(goal)
    neg_goal_pred.negate()
    neg_goal_stmt = Statement(str(neg_goal_pred))

    proved = FOL_Resolution(KB, KB_HASH, neg_goal_stmt, verbose=True)

    print("\nResult: Query 'John likes peanuts' is", "TRUE (proved)" if proved else "FALSE (not proved)")

if __name__ == "__main__":
    main()


Sanjana Srinivas-1BM23CS301

Initial KB clauses (including negated query):
   Alive(x)|Killed(x)
   ~Alive(x)|~Killed(x)
   ~Likes(John,Peanuts)
   Food(Apple)
   Alive(Anil)
   Eats(Anil,Peanuts)
   Food(Vegetables)
   Likes(John,x)|~Food(x)
   Eats(Harry,y)|~Eats(Anil,y)
   Food(y)|Killed(y)|~Eats(p,y)

Iteration 1: Derived 13 new clause(s):
    Alive(x)|~Alive(x)
    Eats(Harry,Peanuts)
    Food(Peanuts)|Killed(Peanuts)
    Food(x)|~Alive(x)|~Eats(p,x)
    Food(y)|Killed(y)|~Eats(Anil,y)
    Food(y)|~Alive(y)|~Eats(p,y)
    Killed(x)|Likes(John,x)|~Eats(p,x)
    Killed(x)|~Killed(x)
    Killed(y)|Likes(John,y)|~Eats(p,y)
    Likes(John,Apple)
    Likes(John,Vegetables)
    ~Food(Peanuts)
    ~Killed(Anil)

Iteration 2: Derived 23 new clause(s):
    Alive(x)
    Food(Anil)|~Eats(Anil,Anil)
    Food(Anil)|~Eats(p,Anil)
    Food(Peanuts)|~Alive(Peanuts)
    Food(x)|Killed(x)|~Eats(Anil,x)
    Food(x)|Killed(x)|~Eats(p,x)
    Food(x)|~Alive(x)|~Eats(Anil,x)
    Food(x)|~Eats(p,x)
    Fo